In [20]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from notebookutils import mssparkutils
from datetime import datetime
from pyspark.sql.functions import explode, lit, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, ArrayType, NullType


StatementMeta(, 91fadfb2-f4c2-4b0e-b89d-46abf4f4174a, 22, Finished, Available, Finished)

In [21]:
schemaJson = StructType([
    StructField("responses", ArrayType(
        StructType([
            StructField("name", StringType(), True),
            StructField("httpStatusCode", IntegerType(), True),
            StructField("headers", StructType([
                StructField("Pragma", StringType(), True),
                StructField("x-ms-failure-cause", StringType(), True),
                StructField("x-ms-request-id", StringType(), True),
                StructField("x-ms-correlation-request-id", StringType(), True),
                StructField("x-ms-routing-request-id", StringType(), True),
                StructField("Strict-Transport-Security", StringType(), True),
                StructField("X-Content-Type-Options", StringType(), True),
                StructField("X-Cache", StringType(), True),
                StructField("X-MSEdge-Ref", StringType(), True),
                StructField("Cache-Control", StringType(), True),
                StructField("Date", StringType(), True)
            ]), True),
            StructField("content", StructType([
                StructField("error", StructType([
                    StructField("code", StringType(), True),
                    StructField("message", StringType(), True)
                ]), True),
                StructField("value", ArrayType(
                    StructType([
                        StructField("name", StructType([
                            StructField("value", StringType(), True),
                            StructField("localizedValue", StringType(), True)
                        ]), True),
                        StructField("unit", StringType(), True),
                        StructField("currentValue", IntegerType(), True),
                        StructField("limit", IntegerType(), True),
                        StructField("id", StringType(), True)
                    ])
                ), True)
            ]), True),
            StructField("contentLength", IntegerType(), True)
        ])
    ), True)
])

StatementMeta(, 91fadfb2-f4c2-4b0e-b89d-46abf4f4174a, 23, Finished, Available, Finished)

In [ ]:
def readQuotaJsonFile(filePath, current_date_int):
    fileName = filePath.split('/')[-1]
    SubscriptionKey = int(fileName.split('_')[1])
    df = spark.read.option("multiline", "true").schema(schemaJson).json(filePath)
    df = df.select(explode(df.responses).alias('col'))
    df= df.select("col.name", explode("col.content.value").alias("value"))
    df = df.select(col("name").alias("RegionId"), col("value.currentValue").cast("int").alias("Usage"), col("value.limit").cast("int").alias("Limit"))
    df = df.withColumn("DateKey", lit(current_date_int)).withColumn("SubscriptionKey", lit(SubscriptionKey))
    return df

In [ ]:
pathQuotas = "Files/Data/Quotas"
quotasListFiles = mssparkutils.fs.ls(pathQuotas)

In [2]:
current_date = int(datetime.now().strftime("%Y%m%d"))

#Clean the date if exists in case of multiple run in the same day. We only keep 1 snapshot per day 
if spark.catalog.tableExists("Quota_fabric"):
     print("Table exists, snapshot will be clean.")
     spark.sql(f"DELETE FROM Quota_fabric WHERE DateKey = {str(current_date)}")

StatementMeta(, 91fadfb2-f4c2-4b0e-b89d-46abf4f4174a, 4, Finished, Available, Finished)

In [ ]:
for Quotafile in quotasListFiles:
    if Quotafile.isFile and Quotafile.name.endswith(".json"):
        print(f"integrate quota data from file {Quotafile.name}")
        df_quota = readQuotaJsonFile(Quotafile.path, current_date)
        df_quota.write.mode("append").saveAsTable("Quota_fabric")

print("Load Done")

#Clean raw files
for Quotafile in quotasListFiles:
    if Quotafile.isFile and Quotafile.name.endswith(".json"):
        mssparkutils.fs.rm(Quotafile.path)

print("Clean Done")